In [13]:
import pandas as pd
import string
import re
import time

from tqdm import tqdm
from tqdm.auto import tqdm

from deep_translator import GoogleTranslator
from langdetect import detect

from nltk.stem.porter import *
from nltk.corpus import stopwords  
from nltk.tokenize import word_tokenize

# Laad de data in

In [2]:
df = pd.read_csv("../data/uploads.csv").drop("Unnamed: 0", axis=1)

In [3]:
df2 = pd.read_csv("../data/channels.csv").drop("Unnamed: 0", axis=1)[["channel", "channel_description", "keywords"]]

In [4]:
df = pd.merge(df, df2, left_on="channel", right_on="channel")

In [5]:
df.head()

,channel,video_id,title,description,conspiracy,transcript,channel_description,keywords
0,UCgzqRRDGThOlH4EHaqSXXPA,Eve8U63jOCA,16th January Update,www.simonparkes.org\n\nTelegram:\nhttps://t.me...,True,hello and just to let you know that i'm doing ...,NaN,NaN
1,UCgzqRRDGThOlH4EHaqSXXPA,T24xqK23NPk,13th January Update,Bitchute Channel:\nhttps://www.bitchute.com/si...,True,hello and welcome to another update it's the 1...,NaN,NaN
2,UCgzqRRDGThOlH4EHaqSXXPA,IJAR8FYdDcE,12th January Second Update Current News,To Join Connecting Consciousness:\nhttps://con...,True,hello and just a another quick update just to ...,NaN,NaN
3,UCgzqRRDGThOlH4EHaqSXXPA,NJej_CUkV8E,11th January Second Update Current News,www.simonparkes.org\n\nTo Join Connecting Cons...,True,hello uh just a very very very quick update it...,NaN,NaN
4,UCgzqRRDGThOlH4EHaqSXXPA,as9IEvaC9XA,11th January Update Current News,www.simonparkes.org\n\nTo Join Connecting Cons...,True,hello and welcome to a sort of flying update i...,NaN,NaN


# Data opschonen

In [6]:
df["conspiracy"].value_counts()

False    43528
True     22156
Name: conspiracy, dtype: int64

In [7]:
# Balanceer de classes
nc = df[df["conspiracy"] == False].sample(22156)
c = df[df["conspiracy"] == True]

equal = pd.concat([nc, c]).reset_index().drop("index", axis=1)
equal.head()

,channel,video_id,title,description,conspiracy,transcript,channel_description,keywords
0,UCyuAKnN3g2fZ7_R9irgEUZQ,SECdW8n4ZGw,How Abraham Lincoln Came to Power,How did Abraham Lincoln come to power? This vi...,False,I’m back. Abraham Lincoln was America's 16th p...,I create video essays about history. If you li...,"history ""animated history"" ""history videos"" ""h..."
1,UCAtdV6VWa02UpCB2YHuVmHg,jSXIrZx8lI4,Thanksgiving Celebration Announcement - Unio...,"On November 21st, 2020 we’ll be serving a Than...",False,our world has changed the way we approach each...,Union Rescue Mission (URM) is a nonprofit orga...,homelessness homeless los angeles LA skid row ...
2,UCASQzDb4SUC0gZNPbwBubIQ,s0YjJVyyVjI,UNC Health's mobile clinic aims to reduce raci...,In their efforts to equitably distribute COVID...,False,We have just found that when the word is\ngett...,"News from The News & Observer, serving Chapel ...","""Triangle news"" ""Raleigh news"" ""North Carolina..."
3,UC_giJ3xlEL9jUF1YfJdzzuQ,VMHCvdRdq10,Establishing Impurity Acceptance Criteria as P...,Submit questions on this poster to DMFWorkshop...,False,- [Yongjun] Welcome.\nMy name is Yongjun Gao. ...,The FDA is responsible for protecting the publ...,FDA recall drug safety device vaccine food US ...
4,UCT8RMFbTJV5ILaVykrluOQg,KKiR4FWOZ4w,Ten panda cubs make public debut ahead of Spri...,Ten panda cubs born in 2020 at a breeding base...,False,NaN,We strive to provide global audiences with new...,CCTVEnglish CCTVNews CCTV英文 CCTV 中央电视台 CCTV4 C...


In [8]:
# Vul lege cellen en lowercase
equal["transcript"] = equal["transcript"].fillna("")
equal["title"] = equal["title"].fillna("")
equal["description"] = equal["description"].fillna("")
equal["keywords"] = equal["keywords"].fillna("")
equal["channel_description"] = equal["channel_description"].fillna("")

equal["transcript"] = equal["transcript"].str.lower()
equal["title"] = equal["title"].str.lower()
equal["description"] = equal["description"].str.lower()
equal["keywords"] = equal["keywords"].str.lower()
equal["channel_description"] = equal["channel_description"].str.lower()

In [9]:
# Voeg titel, beschrijving en transcript samen tot één kolom
equal["full_text"] = equal["title"] + " " + equal["description"] + " " + equal["transcript"]\
                     + " " + equal["channel_description"] + " " + equal["keywords"]

equal = equal.drop(["title", "description", "transcript", "keywords", "channel_description"], axis = 1)

In [14]:
def translate_text(text):
    # Detect language
    lang = detect(text)
    # If the text is not in English, translate it, otherwise just return
    if lang != "en":
        #The translator only handles texts of less than 5000 characters, so we have to split the text
        if len(text) >= 5000:
            text_split = [text[start:start+4999] for start in range(0, len(text), 4999)]
            try:
                return ''.join([GoogleTranslator(source="auto", target='en').translate(txt) for txt in text_split])
            # Deal with connections stutters
            except Exception as e:
                print(e)
                # Wait three seconds to let the connection stablize
                time.sleep(3)
                # Try again, and if it still doesn't work, return the plain text
                try:
                    return ''.join([GoogleTranslator(source="auto", target='en').translate(txt) for txt in text_split])
                except:
                    return text
                
        else:
            # Deal with connection issues
            try:
                return GoogleTranslator(source=lang, target="en").translate(text)
            except Exception as e:
                print(e)
                time.sleep(3)
                try:
                    return GoogleTranslator(source=lang, target="en").translate(text)
                except:
                    return text
    
    else:
        return text

In [15]:
tqdm.pandas()
# Vertaal alle text naar het Engels
equal['full_text'] = equal['full_text'].progress_apply(lambda txt: translate_text(txt))

  0%|          | 0/44312 [00:00<?, ?it/s]

Request exception can happen due to an api connection error. Please check your connection and try again
Request exception can happen due to an api connection error. Please check your connection and try again
Request exception can happen due to an api connection error. Please check your connection and try again


In [23]:
# Remove emoji's and other weird characters
equal["full_text"] = equal["full_text"].progress_apply(lambda text: text.encode("ascii", "ignore").decode())

  0%|          | 0/44312 [00:00<?, ?it/s]

In [25]:
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))

In [26]:
def stem_text(text):
    stripped = [word.strip(string.punctuation) for word in word_tokenize(text) if word not in stop_words]
    return " ".join([stemmer.stem(word) for word in stripped if word])

In [27]:
# Stem transcripts en verwijder stopwords
equal['full_text'] = equal['full_text'].progress_apply(lambda text: stem_text(text))

  0%|          | 0/44312 [00:00<?, ?it/s]

In [28]:
equal["full_text"].fillna("", inplace=True)

In [29]:
equal.to_csv("training_videos.csv")